In [1]:
import pandas as pd
import numpy as np

# SPSS Loader and Editor

Prepares our SPSS file using these standards (https://docs.google.com/document/d/1LZ_i9fHxzdG6w6_Ie7i7wYuiAET9Ubc-SEA3mEiOSDs/edit?tab=t.0)

This notebook loads an SPSS file and then allows the user to change it

In [2]:
# from google.colab import drive
# Mount Google Drive
# drive.mount('/content/drive')
!pip install pyreadstat

  Using cached pyreadstat-1.2.8-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.0 kB)
Using cached pyreadstat-1.2.8-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.9 MB)


In [13]:
!pip install openpyxl

# Load and Edit STUDENT IBM SPSS File

In [3]:
import boto3
import os
import pyreadstat

# Define local file path
#local_file_path_student_miss = "/content/drive/My Drive/PISA_2022_Background_questionnaire_STUDENT_filtered_recode_miss.csv"
local_file_path_student_miss = "PISA_2022_Background_questionnaire_edited.SAV"  # Change as needed

# Define S3 details
bucket_name = "sagemaker-us-west-2-986030204467"
file_key = "capstone/testfiles/PISA_2022_Background_questionnaire_edited.SAV"

# AWS credentials are usually stored in ~/.aws/credentials or IAM roles (if running on AWS services)
s3_client = boto3.client("s3")

# Check if the file exists locally
if os.path.exists(local_file_path_student_miss):
    print(f"📂 Loading data from local file... {local_file_path_student_miss}")
    student_dataset, student_meta = pyreadstat.read_sav(local_file_path_student_miss)
    print("Load completed")
else:
    print("☁️ Downloading data from S3...")
    
    # Create S3 client
    s3_client = boto3.client("s3")

    # Download the file from S3
    response = s3_client.get_object(Bucket=bucket_name, Key=file_key)

    # Read the file into pandas DataFrame
    student_dataset, student_meta = pyreadstat.read_sav(response["Body"])

    # Save a local copy for future use
    student_dataset.to_sav(local_file_path_student_miss, index=False)
    print(f"✅ File saved locally as {local_file_path_student_miss}")

📂 Loading data from local file... PISA_2022_Background_questionnaire_edited.SAV
Load completed


# Add -999 as a Categorical Column

If a categorical column is 'missing' in SPSS, we need to make is something that is NaN

In [4]:
# Identify categorical columns
categorical_cols = student_dataset.select_dtypes(include=['category']).columns

# Add -999 as a new category
for col in categorical_cols:
    student_dataset[col] = student_dataset[col].cat.add_categories([-999])

# Replace all NaN's with -999

CSV's need the 'missing values' from SAV to be 'something', we will use -999

In [ ]:
# Now replace NaN with -999
student_dataset.fillna(-999, inplace=True)

# Check a few variables to see if they now have -999's

In [6]:
student_dataset['ST021Q01TA'].head(10)

0   -999.0
1   -999.0
2      6.0
3   -999.0
4   -999.0
5   -999.0
6   -999.0
7   -999.0
8   -999.0
9   -999.0
Name: ST021Q01TA, dtype: float64

LANGTEST_PAQ has a lot of 'System Missing' variables which are .'s (dots)

In [10]:
student_dataset['LANGTEST_PAQ'].head(10)

0   -999.0
1   -999.0
2   -999.0
3   -999.0
4   -999.0
5   -999.0
6   -999.0
7   -999.0
8   -999.0
9   -999.0
Name: LANGTEST_PAQ, dtype: float64

In [20]:
student_dataset['ST322Q01JA'].head(10)

0      5.0
1   -999.0
2   -999.0
3      3.0
4      3.0
5      3.0
6      5.0
7      1.0
8   -999.0
9   -999.0
Name: ST322Q01JA, dtype: float64

In [23]:
student_dataset.loc[student_dataset['ST322Q01JA'].isin([6, 6.0]), ['ST322Q01JA']]

,ST322Q01JA
19,6.0
21,6.0
36,6.0
53,6.0
72,6.0
...,...
613718,6.0
613719,6.0
613731,6.0
613739,6.0


# Edit the file and remove unneeded columns per Selenes XLSX sheet

In this case we use 

#### 1. Download Student Variables Inclusion/Exclusion Sheet

In [27]:
# Define the Google Sheets export URL
# Variables to include - Students.xlsx
sheet_url = "https://docs.google.com/spreadsheets/d/1rb0AVCWQAEQ-c5vYfKn1aHtmK9bjvq3u/export?format=xlsx"

# Read the Excel file directly from the URL
xls = pd.ExcelFile(sheet_url)

#### 2. Check Dataframe Shape before editing (should have 1280 columns)

In [17]:
student_dataset.shape

(613744, 1280)

#### 3. Use XLS Sheet to Include/Exclude Columns

In [48]:
# Read the Exclude sheet
exclude_df = pd.read_excel(xls, sheet_name="Exclude")

# Read the main data sheet (assuming it's the first sheet)
include_df = pd.read_excel(xls, sheet_name=xls.sheet_names[0])

# Filter columns to keep based on the Exclude sheet conditions
columns_to_keep = exclude_df[
    (exclude_df["Include"] == "x") & (exclude_df["Reason for exclusion"] == "Can use WLE")
]["NAME"].tolist()

# Identify columns where "Include" = 'O' and add them to columns_to_keep
columns_to_keep += include_df[
    (include_df["Include"] == "o") & (include_df["Reasons for inclusion"] != "WLE")
]["NAME"].tolist()

# Filter dataframe columns
df_filtered = student_dataset[columns_to_keep]

In [49]:
def count_nullables_before(df, include_df):
    """
    Count occurrences of nullable values before replacement.

    :param df: Pandas DataFrame (student dataset)
    :param include_df: DataFrame containing the 'Include' sheet
    :return: Dictionary with counts of nullable values per column
    """
    count_dict = {}

    for _, row in include_df.iterrows():
        col = row['NAME']
        nullables = parse_nullable_values(row['nullables'])

        if pd.notna(col) and pd.notna(row['nullables']) and col in df.columns:
            count_dict[col] = df[col].isin(nullables).sum()

    return count_dict

# Count nullables before replacement
nullables_before = count_nullables_before(student_dataset, include_df)
nullables_before

{'ST322Q01JA': 39491,
 'ST322Q02JA': 25559,
 'ST322Q03JA': 17624,
 'ST322Q04JA': 41568,
 'ST322Q06JA': 51716,
 'ST322Q07JA': 37880,
 'LANGN': 0,
 'IC184Q01JA': 13423,
 'IC184Q02JA': 12960,
 'IC184Q03JA': 23619,
 'IC184Q04JA': 30010,
 'WB161Q01HA': 12339}

#### 4. Replace all 'other nullables' with -999

In [50]:
def parse_nullable_values(value):
    """Convert nullable column values into lists."""
    if pd.isna(value):  # If NaN, return empty list
        return []
    if isinstance(value, str):  # Check if the value is a string (like '998,999')
        return [int(v.strip()) for v in value.split(",")]  # Convert CSV string to list of ints
    return [value]  # If it's a single number, wrap it in a list

def replace_values_with_neg999(df, include_df):
    """
    Replace specified nullable values in columns based on 'Include' sheet.

    :param df: Pandas DataFrame (student dataset)
    :param include_df: DataFrame containing the 'Include' sheet
    :return: Modified DataFrame
    """
    # Build dictionary of columns and their nullable values
    replace_dict = {
        row['NAME']: parse_nullable_values(row['nullables'])
        for _, row in include_df.iterrows()
        if pd.notna(row['NAME']) and pd.notna(row['nullables'])  # Ensure columns and nullables are valid
    }

    # Apply replacements using .loc to avoid SettingWithCopyWarning
    for col, values in replace_dict.items():
        if col in df.columns:
            df.loc[:, col] = df[col].replace(values, -999)
    
    return df

# Apply replacements based on 'Include' sheet
df_filtered = replace_values_with_neg999(df_filtered, include_df)

In [51]:
df_filtered.loc[student_dataset['ST322Q06JA'].isin([6, 6.0]), ['ST322Q01JA']]

,ST322Q01JA
19,-999.0
21,-999.0
36,-999.0
37,-999.0
52,5.0
...,...
613730,-999.0
613735,1.0
613737,5.0
613739,-999.0


In [52]:
def count_neg999_after(df, include_df, nullables_before):
    """
    Count occurrences of -999 after replacement, but only for columns that had nullables before.

    :param df: Pandas DataFrame (student dataset)
    :param include_df: DataFrame containing the 'Include' sheet
    :param nullables_before: Dictionary of nullable counts before replacement
    :return: Dictionary with counts of -999 per column (only for nullables)
    """
    count_dict = {}

    for col in nullables_before.keys():  # Only check columns that had nullables before
        if col in df.columns:
            count_dict[col] = (df[col] == -999).sum()

    return count_dict

# Count -999 occurrences after replacement (only for columns that had nullables before)
neg999_after = count_neg999_after(student_dataset, include_df, nullables_before)
neg999_after

{'ST322Q01JA': 166041,
 'ST322Q02JA': 167654,
 'ST322Q03JA': 168714,
 'ST322Q04JA': 168657,
 'ST322Q06JA': 168887,
 'ST322Q07JA': 168107,
 'LANGN': 0,
 'IC184Q01JA': 284814,
 'IC184Q02JA': 286654,
 'IC184Q03JA': 285610,
 'IC184Q04JA': 286157,
 'WB161Q01HA': 495107}

#### 2. Check Dataframe Shape AFTER editing (should have 735 columns)

In [53]:
df_filtered.shape

(613744, 735)

# Remap Countries

In [54]:
# Define the country code to name mapping
country_mapping = {
    "ALB": "Albania", "ARE": "United_Arab_Emirates", "ARG": "Argentina", "AUS": "Australia",
    "AUT": "Austria", "BEL": "Belgium", "BGR": "Bulgaria", "BRA": "Brazil", "BRN": "Brunei_Darussalam",
    "CAN": "Canada", "CHE": "Switzerland", "CHL": "Chile", "COL": "Colombia", "CRI": "Costa_Rica",
    "CZE": "Czech_Republic", "DEU": "Germany", "DNK": "Denmark", "DOM": "Dominican_Republic",
    "ESP": "Spain", "EST": "Estonia", "FIN": "Finland", "FRA": "France", "GBR": "United_Kingdom",
    "GEO": "Georgia", "GRC": "Greece", "GTM": "Guatemala", "HRV": "Croatia", "HUN": "Hungary",
    "IDN": "Indonesia", "IRL": "Ireland", "ISL": "Iceland", "ISR": "Israel", "ITA": "Italy",
    "JAM": "Jamaica", "JOR": "Jordan", "JPN": "Japan", "KAZ": "Kazakhstan", "KHM": "Cambodia",
    "KOR": "Korea", "KSV": "Kosovo", "LTU": "Lithuania", "LVA": "Latvia", "MAR": "Morocco",
    "MDA": "Republic_of_Moldova", "MEX": "Mexico", "MKD": "North_Macedonia", "MLT": "Malta",
    "MNE": "Montenegro", "MNG": "Mongolia", "MYS": "Malaysia", "NLD": "Netherlands", "NOR": "Norway",
    "NZL": "New_Zealand", "PAN": "Panama", "PER": "Peru", "PHL": "Philippines", "POL": "Poland",
    "PRT": "Portugal", "PRY": "Paraguay", "PSE": "Palestinian_Authority", "QAT": "Qatar",
    "QCY": "Cyprus", "ROU": "Romania", "SAU": "Saudi_Arabia", "SGP": "Singapore", "SLV": "El_Salvador",
    "SRB": "Serbia", "SVK": "Slovak_Republic", "SVN": "Slovenia", "SWE": "Sweden", "TAP": "Taiwan",
    "THA": "Thailand", "TUR": "Türkiye", "URY": "Uruguay", "USA": "United_States", "UZB": "Uzbekistan",
    "VNM": "Vietnam"
}

# Apply the mapping to the 'CNT' column
student_dataset.loc[:, "CNT"] = student_dataset["CNT"].replace(country_mapping)

In [57]:
student_dataset["CNT"].unique()

array(['Albania', 'QAZ', 'Argentina', 'Australia', 'Austria', 'Belgium',
       'Brazil', 'Brunei_Darussalam', 'Bulgaria', 'Cambodia', 'Canada',
       'Chile', 'Taiwan', 'Colombia', 'Costa_Rica', 'Croatia',
       'Czech_Republic', 'Denmark', 'Dominican_Republic', 'El_Salvador',
       'Estonia', 'Finland', 'France', 'Georgia', 'Palestinian_Authority',
       'Germany', 'Greece', 'Guatemala', 'HKG', 'Hungary', 'Iceland',
       'Indonesia', 'Ireland', 'Israel', 'Italy', 'Kosovo', 'Jamaica',
       'Japan', 'Kazakhstan', 'Jordan', 'Korea', 'Latvia', 'Lithuania',
       'MAC', 'Malaysia', 'Malta', 'Mexico', 'Mongolia',
       'Republic_of_Moldova', 'Montenegro', 'Morocco', 'Netherlands',
       'New_Zealand', 'Norway', 'Panama', 'Paraguay', 'Peru',
       'Philippines', 'Poland', 'Portugal', 'Qatar', 'Romania',
       'Saudi_Arabia', 'Serbia', 'Singapore', 'Slovak_Republic',
       'Vietnam', 'Slovenia', 'Spain', 'Sweden', 'Switzerland',
       'Thailand', 'United_Arab_Emirates', 'Türki

In [55]:
df_filtered.head(5)

,ST250Q01JA,ST250Q02JA,ST250Q03JA,ST250Q04JA,ST250Q05JA,ST250D06JA,ST250D07JA,ST251Q01JA,ST251Q02JA,ST251Q03JA,...,WB177Q02HA,WB177Q03HA,WB177Q04HA,WB032Q01NA,WB032Q02NA,WB031Q01NA,EXERPRAC,STUBMI,WORKPAY,WORKHOME
0,-999.0,1.0,-999.0,-999.0,-999.0,9999997,9999997,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,-999.0,0.0,10.0
1,2.0,2.0,2.0,1.0,2.0,9999997,9999997,1.0,2.0,1.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2,1.0,1.0,1.0,1.0,1.0,9999997,9999997,2.0,3.0,3.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,0.0,0.0
3,1.0,1.0,2.0,1.0,1.0,9999997,9999997,1.0,1.0,1.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,10.0,-999.0,0.0,10.0
4,1.0,1.0,1.0,1.0,1.0,9999997,9999997,3.0,1.0,2.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,2.0,-999.0,0.0,4.0


# Remove non-nationally representative countries

In [58]:
# Define the values to remove
remove_countries = ["QUR", "HKG", "MAC", "QAZ"]

# Filter out the unwanted rows
df_filtered = df_filtered[~df_filtered["CNT"].isin(remove_countries)]

In [59]:
df_filtered.shape

(591857, 735)

# Write out our new file

In [60]:
df_filtered.to_csv('student_filtered_and_edited.csv')